# Create a virtual Zarr dataset from the individual AORC-OWP JSON files

In [ ]:
import fsspec
import xarray as xr
import ujson   # fast json
from pathlib import Path
from kerchunk.combine import MultiZarrToZarr, auto_dask, JustLoad

In [ ]:
fs_write = fsspec.filesystem('file')

##### Use a helper function specific to this Nebari deployment to start Dask cluster

In [ ]:
from dask.distributed import LocalCluster, Client

cluster = LocalCluster(threads_per_worker=1)
client = Client(cluster)
client

In [ ]:
json_dir = '/caldera/hytest_scratch/scratch/rsignell/json_files'

In [ ]:
json_list = fs_write.glob(f'{json_dir}/*.json')

In [ ]:
json_list[0]

In [ ]:
so = dict(mode='rb', default_fill_cache=False, default_cache_type='first')

## merge individual jsons into single combined JSON (serial process)

#### merge individual jsons into single combined JSON in parallel using Dask

In [ ]:
worker_max=80

In [ ]:
%%time
d = auto_dask(
    json_list,
    single_driver=JustLoad,
    single_kwargs={"storage_options": {}},
    mzz_kwargs={"concat_dims": ["time"]},
    n_batches=worker_max,   # give one batch to each worker
#    remote_protocol="s3",
#    remote_options={"requester_pays": True}
)

#### Explore the combined JSON

In [ ]:
%%time
fs = fsspec.filesystem("reference", fo=d, ref_storage_args={'skip_instance_cache':True})
m = fs.get_mapper("")

ds = xr.open_dataset(m, engine="zarr", backend_kwargs={'consolidated':False}, chunks={})

In [ ]:
ds

In [ ]:
ds.APCP_surface

## visualize 

In [ ]:
import hvplot.xarray

In [ ]:
da = ds.APCP_surface.sel(time='2000-02-15 06:00').load()

In [ ]:
da.hvplot.image(x='longitude', y='latitude', geo=True, rasterize=True, cmap='turbo', tiles='OSM')

#### Extract a time series

In [ ]:
%%time
da = ds.APCP_surface.sel(longitude=-115.18, latitude=46.65, method='nearest').load()

In [ ]:
da.hvplot(x='time', grid=True)

write combined JSON to storage

In [ ]:
combined_json = '/caldera/hytest_scratch/scratch/rsignell/sudhir.json'

In [ ]:
%%time
with fs_write.open(combined_json, 'wb') as f:
    f.write(ujson.dumps(d).encode());